<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Cargamos-el-csv-que-vamos-a-usar" data-toc-modified-id="Cargamos-el-csv-que-vamos-a-usar-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cargamos el <code>csv</code> que vamos a usar</a></span></li><li><span><a href="#Definición-de-variables" data-toc-modified-id="Definición-de-variables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Definición de variables</a></span></li><li><span><a href="#Creamos-la-clase" data-toc-modified-id="Creamos-la-clase-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Creamos la clase</a></span><ul class="toc-item"><li><span><a href="#Hacemos-las-llamadas-a-los-métodos-de-la-Clase" data-toc-modified-id="Hacemos-las-llamadas-a-los-métodos-de-la-Clase-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Hacemos las llamadas a los métodos de la Clase</a></span><ul class="toc-item"><li><span><a href="#Iniciamos-la-Clase" data-toc-modified-id="Iniciamos-la-Clase-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Iniciamos la Clase</a></span></li><li><span><a href="#Llamada-a-la-API" data-toc-modified-id="Llamada-a-la-API-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Llamada a la API</a></span></li><li><span><a href="#Limpiamos-los-dataframes" data-toc-modified-id="Limpiamos-los-dataframes-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Limpiamos los dataframes</a></span></li><li><span><a href="#Juntamos-los-dataframes" data-toc-modified-id="Juntamos-los-dataframes-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Juntamos los dataframes</a></span></li><li><span><a href="#Chequeamos-como-están-los-datos-de-&quot;limpios&quot;" data-toc-modified-id="Chequeamos-como-están-los-datos-de-&quot;limpios&quot;-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>Chequeamos como están los datos de "limpios"</a></span></li></ul></li></ul></li></ul></div>

En este jupyter vamos a hacer lo mismo que en el anterior, pero en este caso lo haremos definiendo una clase que nos permita hacer todo el proceso. 

In [4]:
import pandas as pd
import requests
from datetime import datetime, timedelta

# Cargamos el `csv` que vamos a usar

In [6]:
df = pd.read_csv("files/datos_actualizados.csv", index_col = 0)
df.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,1,15,none,0,5,64,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
1,6,2,15,none,0,6,55,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2


# Definición de variables 

Definimos las variables que vamos a necesitar para hacer las llamadas a cada uno de los métodos de la clase. 

In [7]:
latitud = 40.4165
longitud = -3.70256
producto = "civil"
producto2 = "astro"
ciudad = "Madrid"


# Creamos la clase 

In [23]:
class Extraccion: 
    # primero tenemos que crear el constructor con las variables globales que usaremos. Están son variables que podrán cambiar cada vez que llamemos a la clase. 
    def __init__(self, lat, lon, ciudad):

        # definimos cada una de las variables. Recordamos que tenemos que usar el método .self para definirlas y que la clase entienda que las podremos usar en otros métodos de la clase. 
        self.lat = lat
        self.lon = lon
        self.ciudad = ciudad

    # definimos nuestro primer método. El que nos hará la llamada a la API
    # Si comparamos, veremos que la función es exactamente igual a la que hicimos en la clase anterior, solo que con unos pequeños cambios. En este caso incluimos el parámetro self
    def llamada_API(self, producto):
        
        # producto es una variable nueva, por lo que lo primero que tenemos que hacer es definirla
        self.producto = producto
    
        # hacemos la llamada  a la API
        # Tendremos que poner el self delante de long y lat ya que son variables definidas en el constructor. El self podríamos decir que es como el camino de baldosas 
        # amarillas que indica al método de donde tiene que sacar esas variables
        url = f'http://www.7timer.info/bin/api.pl?lon=-{self.lon}&lat={self.lat}&product={producto}&output=json'

        response = requests.get(url=url)
        codigo_estado = response.status_code
        razon_estado = response.reason
        if codigo_estado == 200:
            print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

        # convertimos los resultados en un dataframe: 
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        return df
    
    # definimos un nuevo método, el de limpiar los datos obtenidos de la llamada a la API para el producto civil. 
    def limpiar_civil(self, df): 

        # lo primero que tenemos que hacer es crear la columna de fecha en el dataframe nuevo 
        # En este caso será la fecha del día de hoy. 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')

        # creamos la nueva columna
        df["fecha"] = hoy

        # de nuevo usamos el self para crear las nuevas columnas en función de los parámetros pasados al definir la clase. Recordamos, el caminito de baldosas amarillas.
        df["latitud"] = self.lat
        df["longitud"] = self.lon
        df["ciudad"] = self.ciudad
        return df
    
    
    # creamos una clase para limpiar el resultado de la llamada a la API para el producto astro
    def limpiar_astro(self, df):

        #seleccionamos solo las columnas que nos interesan
        df = df[["seeing", "transparency", "timepoint"]]

        # creamos la columna de fecha: 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        df["fecha"] = hoy


        # insertamos las columnas de la localidad
    
        df["ciudad"] = self.ciudad

        return df
    
    # de nuevo, hacemos lo mismo que en la lección anterior. 
    def juntar_dfs(self, df_completo, df_civil, df_visibilidad): 

        df_hoy = pd.merge(df_civil , df_visibilidad , on=['fecha', "timepoint", "ciudad"], how = "right")
        print("El df de hoy es ", df_hoy.columns)
        print("-----------------------------------------")
        print("El df de completo es ", df_completo.columns)
        # lo primero que hacemos es concatenar los dataframes con la información general. df_completo y df_civil
        df_completo = pd.concat([df_completo, df_hoy], axis = 0)

        
        # guardamos los datos
        df_completo.to_pickle('files/datos_actualizados.pkl')
        df_completo.to_csv('files/datos_actualizados.csv')

        return df_completo
    
    

    def chequear_datos(self, df): 
    
        print("Las columnas son:", "\n")
        print(list(df.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df.isnull().sum() / df.shape[0]) *  100)

    def limpiar_dataframe(self, df, lista_columnas = []): 

        #convertimos la fecha a datetime
        df["fecha"] = pd.to_datetime(df["fecha"])

        # reemplazamos los nulos de las columnas por la media
         # lista de columnas en las que queremos reemplazar los nulos
        df[lista_columnas]=df[lista_columnas].fillna(df.mean())

        
        # renombrar columnas
        
        df.rename(columns = {"ciudad_x": "ciudad"}, inplace = True)

        # quitar % 
        df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)

        # guardamos los datos una vez limpios
        df.to_pickle('files/datos_actualizados.pkl')
        df.to_csv('files/datos_actualizados.csv')

        return df


## Hacemos las llamadas a los métodos de la Clase 

### Iniciamos la Clase 
´
Recordamos que nuestro método constructor recibía tres parámetros. 

In [25]:
api = Extraccion(latitud, longitud, ciudad)

In [26]:
api

### Llamada a la API 

1️⃣ Llamamos al producto "civil"

In [27]:
df_civil = api.llamada_API("civil")
df_civil.head(2)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed
0,3,1,15,none,0,8,40%,clearday,NW,2
1,6,1,10,none,0,13,38%,clearday,NW,2


2️⃣ Llamamos al producto "astro"

In [28]:
df_astro = api.llamada_API("astro")
df_astro.head(2)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,seeing,transparency,lifted_index,rh2m,temp2m,prec_type,wind10m.direction,wind10m.speed
0,3,1,2,2,15,4,8,none,NW,2
1,6,1,2,2,10,3,13,none,NW,2


### Limpiamos los dataframes

In [29]:
df_civil = api.limpiar_civil(df_civil)
df_civil.head(2)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad
0,3,1,15,none,0,8,40%,clearday,NW,2,2023-01-10,40.4165,-3.70256,Madrid
1,6,1,10,none,0,13,38%,clearday,NW,2,2023-01-10,40.4165,-3.70256,Madrid


In [30]:
df_astro = api.limpiar_astro(df_astro)
df_astro.head(2)

/tmp/ipykernel_785/3350619388.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["fecha"] = hoy
/tmp/ipykernel_785/3350619388.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ciudad"] = self.ciudad


,seeing,transparency,timepoint,fecha,ciudad
0,2,2,3,2023-01-10,Madrid
1,2,2,6,2023-01-10,Madrid


In [31]:
df_civil.head(1)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad
0,3,1,15,none,0,8,40%,clearday,NW,2,2023-01-10,40.4165,-3.70256,Madrid


In [32]:
df_astro.head(1)

,seeing,transparency,timepoint,fecha,ciudad
0,2,2,3,2023-01-10,Madrid


In [33]:
df.head(1)

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,1,15,none,0,5,64,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2


### Juntamos los dataframes 

In [34]:
df_completo = api.juntar_dfs(df, df_civil, df_astro)
df_completo.head(2)

El df de hoy es  Index(['timepoint', 'cloudcover', 'lifted_index', 'prec_type', 'prec_amount',
       'temp2m', 'rh2m', 'weather', 'wind10m.direction', 'wind10m.speed',
       'fecha', 'latitud', 'longitud', 'ciudad', 'seeing', 'transparency'],
      dtype='object')
-----------------------------------------
El df de completo es  Index(['timepoint', 'cloudcover', 'lifted_index', 'prec_type', 'prec_amount',
       'temp2m', 'rh2m', 'weather', 'wind10m.direction', 'wind10m.speed',
       'fecha', 'latitud', 'longitud', 'ciudad', 'seeing', 'transparency'],
      dtype='object')


,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,1,15,none,0,5,64,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
1,6,2,15,none,0,6,55,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2


In [35]:
df_completo.isnull().sum()

timepoint            0
cloudcover           0
lifted_index         0
prec_type            0
prec_amount          0
temp2m               0
rh2m                 0
weather              0
wind10m.direction    0
wind10m.speed        0
fecha                0
latitud              0
longitud             0
ciudad               0
seeing               0
transparency         0
dtype: int64

### Chequeamos como están los datos de "limpios"

In [36]:
api.chequear_datos(df_completo)

Las columnas son: 

['timepoint', 'cloudcover', 'lifted_index', 'prec_type', 'prec_amount', 'temp2m', 'rh2m', 'weather', 'wind10m.direction', 'wind10m.speed', 'fecha', 'latitud', 'longitud', 'ciudad', 'seeing', 'transparency']
-----------------------------------------
Los tipos de datos que tenemos son: 

timepoint              int64
cloudcover             int64
lifted_index           int64
prec_type             object
prec_amount            int64
temp2m                 int64
rh2m                  object
weather               object
wind10m.direction     object
wind10m.speed          int64
fecha                 object
latitud              float64
longitud             float64
ciudad                object
seeing                 int64
transparency           int64
dtype: object
-----------------------------------------
El porcentaje de nulos: 

timepoint            0.0
cloudcover           0.0
lifted_index         0.0
prec_type            0.0
prec_amount          0.0
temp2m               0

In [37]:
df_final = api.limpiar_dataframe(df_completo)
df_final.head(2)

/tmp/ipykernel_785/3350619388.py:113: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df[lista_columnas]=df[lista_columnas].fillna(df.mean())
/tmp/ipykernel_785/3350619388.py:113: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df[lista_columnas]=df[lista_columnas].fillna(df.mean())


,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,fecha,latitud,longitud,ciudad,seeing,transparency
0,3,1,15,none,0,5,64,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2
1,6,2,15,none,0,6,55,clearnight,NW,3,2023-01-10,40.4165,-3.70256,Madrid,5,2


Y listo!! Si cada día ejecutaramos este código tendríamos actualizado al día la información meteorológica para la ciudad de Madrid 💪🏽